In [25]:
import tkinter as tk
from tkinter import INSERT, messagebox
from tkinter.ttk import *
import bs4
import requests
import sqlite3

In [ ]:
class takeInputFromWeb():
    def __init__(self, link):
        self.link = link
        
        self.control = False
        self.window = tk.Tk()
        self.window.geometry("{0}x{1}+0+0".format(self.window.winfo_screenwidth(), self.window.winfo_screenheight()))
        self.window.title('Endeksleme ve Arama')
        
        self.nameLabel = tk.Label(self.window,text = 'Gizem Beden - Alperen Ordukaya\n210911175 - 210911050')
        self.nameLabel.pack(padx = 0, pady=0, side=tk.TOP)
        
        self.startCrawlingButton = tk.Button(self.window,command = lambda m = '':self.startCrawlingButtonClicked(), text = 'Start Crawling')
        self.startCrawlingButton.pack(padx = 0, pady = 0, side = tk.TOP)
        
        self.wordsLabel = tk.Label(self.window,text = 'Arama yapılacak kelime ya da kelimeleri giriniz:')
        self.wordsLabel.pack()
        
        self.wordsTextBox = tk.Text(self.window,height=1, width=50)
        self.wordsTextBox.pack()
        
        self.var1 = tk.IntVar()
        self.var2 = tk.IntVar()
        self.var3 = tk.IntVar()
        
        self.wordFrequency = tk.Checkbutton(self.window, text='Word Frequency', onvalue=1, offvalue=0, variable = self.var1)
        self.wordFrequency.pack()
        
        self.inboundLink = tk.Checkbutton(self.window, text='Inbound Link', onvalue=1, offvalue=0, variable = self.var2)
        self.inboundLink.pack()
        
        self.pageRank = tk.Checkbutton(self.window, text='Page Link', onvalue=1, offvalue=0, variable = self.var3)
        self.pageRank.pack()
        
        self.startProcess = tk.Button(self.window,command = lambda m = '':self.inputControl(), text = 'Search')
        self.startProcess.pack(padx=0,pady=0, side = tk.TOP)
        
        self.resultScreen = tk.Text(self.window, height=25, width=175)
        self.resultScreen.pack()
        
        self.window.mainloop()
    
    def inputControl(self):
        if self.control == True:
            if (self.var1.get() == 0 and self.var2.get() == 0) or self.wordsTextBox.get('1.0','end-1c') == '':
                messagebox.showinfo(title = 'ERROR!', message = 'PARAMETER NOT SPECIFIED!')
            elif(self.var3.get() == 1):
                messagebox.showinfo(title = 'ERROR!', message = 'Burası tamamlanamadı!')
            else:
                inputValue = self.wordsTextBox.get('1.0', 'end-1c')
                inputList = list()
                wordFrequency = False
                inboundLink = False
                pageRank = False
                
                if ',' in inputValue:
                    inputList = inputValue.lower().split(',')
                elif '-' in inputValue:
                    inputList = inputValue.lower().split('-')
                else:
                    inputList.append(inputValue)
                    
                if self.var1.get() == 1:
                    kelimeFrekansi = True
                if self.var2.get() == 1:
                    inboundLink = True 
                if self.var3.get() == 1:
                    pageRank = True
                linkList,linkListHead = self.findAllLink()
                self.resultScreen.delete('1.0','end')
                self.resultScreen.insert(INSERT, 'Results:\n')
                self.findWordFromLink(inputList, linkList,linkListHead)
        elif self.control == False:
            messagebox.showinfo(title = 'ERROR!', message = 'Crawling MUST START FIRST!')
    
    def startCrawlingButtonClicked(self):
        if self.control == False:
            messagebox.showinfo(title = 'LAUNCHED!', message = 'Crawling STARTED!')
            self.resultScreen.insert(INSERT, 'You can now search, please enter the above information.')
            self.control = True
            connection = sqlite3.connect('database.db')
            cursor = connection.cursor()
            cursor.execute('''CREATE TABLE IF NOT EXISTS links
                  (link TEXT, head TEXT)''')
            connection.commit()
            cursor.close()
            connection.close()
        
        elif self.control == True:
            messagebox.showinfo(title = 'Already Working!', message = 'CREWING IS ALREADY ACTIVE!')

    
    def addToDatabase(self,link,head):
        sqliteConnection = sqlite3.connect('database.db')
        cursor  = sqliteConnection.cursor()
          
        cursor.execute('INSERT INTO links (link,head) VALUES (?,?)', (link,head))
        sqliteConnection.commit()
        cursor.close()
    
    def takeFromDatabase(self):
        conn = sqlite3.connect('database.db')
        cur = conn.cursor()
        cur.execute("SELECT link,head FROM links")
        rows = cur.fetchall()
        
        connection.commit()
        connection.close()
        
        return rows
        
    def findAllLink(self):
        r = requests.get(self.link)
        source = bs4.BeautifulSoup(r.content, 'lxml')
        
        allLink = list()
        allLink.append(self.link)
        
        allLinkHead = list()
        allLinkHead.append('Anasayfa')
        
        for link in source.find_all('a'):
            newlink = 'https://ois.istinye.edu.tr'+link.get('href')
            allLink.append(newlink)
            allLinkHead.append(link.text)
            
           
        
        return allLink,allLinkHead
                
    def findWordFromLink(self, wordList, linkList,linkListHead):
        zipObject = zip(linkList,linkListHead)
        newList = list()
        timer_1 = 0.0
        timer_2 = 0.0
        
        for word in wordList:
            
            for link,head in zipObject:
                r = requests.get(link)
                source = bs4.BeautifulSoup(r.content, 'lxml')
                for linkWords in source.find_all('td'):
                    if word.lower() in linkWords.text.lower():
                        timer_2 += 1
                        timer_1 += 1
                        newList.append((word,head,link))
                        self.addToDatabase(link,head)
                    
        timer_1 /= len(newList)
        newList  = list(set(newList))
        for word,head,link in newList:
            if self.var1.get() == 1 and self.var2.get() == 0 and self.var3.get() == 0:
                self.resultScreen.insert(INSERT, f'\n\n {timer_1} {word} {head} -> {link}')

            if self.var1.get() == 0 and self.var2.get() == 1 and self.var3.get() == 0:
                self.resultScreen.insert(INSERT, f'\n\n {timer_2} {word} {head} -> {link}')
            
            if self.var1.get() == 0 and self.var2.get() == 0 and self.var3.get() == 1:
                self.resultScreen.insert(INSERT, f'\n\n {timer_2} {word} {head} -> {link}')
                
            if self.var1.get() == 1 and self.var2.get() == 1 and self.var3.get() == 0:
                timer_1 = (timer_1 + timer_2) / 2
                self.resultScreen.insert(INSERT, f'\n\n {timer_2} {word} {head} -> {link}')    
            
takeInputFromWeb('https://ois.istinye.edu.tr/bilgipaketi/eobsakts/ogrenimprogrami/program_kodu/0401001/menu_id/p_38/tip/L/submenuheader/2/ln/tr/print/1')